# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [33]:
import graphlab
import numpy

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('/Users/apple/Desktop/kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1492006458.log


This non-commercial license of GraphLab Create for academic use is assigned to kobecool248@163.com and will expire on March 05, 2018.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.841626     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.897039     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [30]:
list(model_all.coefficients['value'])

[274873.0559504957,
 0.0,
 0.0,
 8468.531086910072,
 24.420720982445214,
 350.0605533860648,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 842.0680348976282,
 20.024722417091304,
 0.0,
 0.0,
 0.0]

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [9]:
for l1_penalty in numpy.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,verbose=False)
    errors=validation['price']-model.predict(validation)
    RSS=sum(errors**2)
    print 'l1_penalty = %e' % l1_penalty
    print 'number of nonzeros = %d' % (model.coefficients['value']).nnz()
    print RSS

l1_penalty = 1.000000e+01
number of nonzeros = 18
6.25766285142e+14
l1_penalty = 3.162278e+01
number of nonzeros = 18
6.25766285362e+14
l1_penalty = 1.000000e+02
number of nonzeros = 18
6.25766286058e+14
l1_penalty = 3.162278e+02
number of nonzeros = 18
6.25766288257e+14
l1_penalty = 1.000000e+03
number of nonzeros = 18
6.25766295212e+14
l1_penalty = 3.162278e+03
number of nonzeros = 18
6.25766317206e+14
l1_penalty = 1.000000e+04
number of nonzeros = 18
6.25766386761e+14
l1_penalty = 3.162278e+04
number of nonzeros = 18
6.25766606749e+14
l1_penalty = 1.000000e+05
number of nonzeros = 18
6.25767302792e+14
l1_penalty = 3.162278e+05
number of nonzeros = 18
6.25769507644e+14
l1_penalty = 1.000000e+06
number of nonzeros = 18
6.25776517727e+14
l1_penalty = 3.162278e+06
number of nonzeros = 18
6.25799062845e+14
l1_penalty = 1.000000e+07
number of nonzeros = 18
6.25883719085e+14


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [11]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [12]:
l1_penalty_values = numpy.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [15]:
for l1_penalty in numpy.logspace(8, 10, num=20):
    model_2= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,verbose=False)
    print 'l1_penalty = %e' % l1_penalty
    print 'number of nonzeros = %d' % (model_2.coefficients['value']).nnz()
    

l1_penalty = 1.000000e+08
number of nonzeros = 18
l1_penalty = 1.274275e+08
number of nonzeros = 18
l1_penalty = 1.623777e+08
number of nonzeros = 18
l1_penalty = 2.069138e+08
number of nonzeros = 18
l1_penalty = 2.636651e+08
number of nonzeros = 17
l1_penalty = 3.359818e+08
number of nonzeros = 17
l1_penalty = 4.281332e+08
number of nonzeros = 17
l1_penalty = 5.455595e+08
number of nonzeros = 17
l1_penalty = 6.951928e+08
number of nonzeros = 17
l1_penalty = 8.858668e+08
number of nonzeros = 16
l1_penalty = 1.128838e+09
number of nonzeros = 15
l1_penalty = 1.438450e+09
number of nonzeros = 15
l1_penalty = 1.832981e+09
number of nonzeros = 13
l1_penalty = 2.335721e+09
number of nonzeros = 12
l1_penalty = 2.976351e+09
number of nonzeros = 10
l1_penalty = 3.792690e+09
number of nonzeros = 6
l1_penalty = 4.832930e+09
number of nonzeros = 5
l1_penalty = 6.158482e+09
number of nonzeros = 3
l1_penalty = 7.847600e+09
number of nonzeros = 1
l1_penalty = 1.000000e+10
number of nonzeros = 1


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [18]:
l1_penalty_min = 2.976351e+09
l1_penalty_max = 3.792690e+09

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [19]:
l1_penalty_values = numpy.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [22]:
for l1_penalty in l1_penalty_values:
    model_3= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,verbose=False)
    print 'l1_penalty = %e' % l1_penalty
    print 'number of nonzeros = %d' % (model_3.coefficients['value']).nnz()
    errors=validation['price']-model_3.predict(validation)
    RSS=sum(errors**2)
    print RSS
    

l1_penalty = 2.976351e+09
number of nonzeros = 10
9.6692560828e+14
l1_penalty = 3.019316e+09
number of nonzeros = 10
9.74019365099e+14
l1_penalty = 3.062281e+09
number of nonzeros = 10
9.81188325006e+14
l1_penalty = 3.105247e+09
number of nonzeros = 10
9.89328293176e+14
l1_penalty = 3.148212e+09
number of nonzeros = 10
9.98783153887e+14
l1_penalty = 3.191177e+09
number of nonzeros = 10
1.00847705074e+15
l1_penalty = 3.234142e+09
number of nonzeros = 10
1.01829866275e+15
l1_penalty = 3.277107e+09
number of nonzeros = 10
1.02824793254e+15
l1_penalty = 3.320073e+09
number of nonzeros = 8
1.03461688584e+15
l1_penalty = 3.363038e+09
number of nonzeros = 8
1.03855471241e+15
l1_penalty = 3.406003e+09
number of nonzeros = 8
1.04323720881e+15
l1_penalty = 3.448968e+09
number of nonzeros = 7
1.04693744459e+15
l1_penalty = 3.491934e+09
number of nonzeros = 7
1.05114759685e+15
l1_penalty = 3.534899e+09
number of nonzeros = 7
1.05599270637e+15
l1_penalty = 3.577864e+09
number of nonzeros = 7
1.0607

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [23]:
m=graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty = 3.448968e+09,verbose=False)

In [25]:
m.get('coefficients')

name,index,value,stderr
(intercept),None,222253.151147,None
bedrooms,None,661.725505133,None
bedrooms_square,None,0.0,None
bathrooms,None,15873.9612122,None
sqft_living,None,32.4102249064,None
sqft_living_sqrt,None,690.114962924,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None


In [31]:
list(m.coefficients['value'])

[222253.15114671466,
 661.7255051333967,
 0.0,
 15873.96121220416,
 32.41022490640613,
 690.1149629237992,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2899.421534576034,
 30.0115803967656,
 0.0,
 0.0,
 0.0]

In [32]:
list(all_features)

['bedrooms',
 'bedrooms_square',
 'bathrooms',
 'sqft_living',
 'sqft_living_sqrt',
 'sqft_lot',
 'sqft_lot_sqrt',
 'floors',
 'floors_square',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']